In [1]:
# Check if python is 3.9.5
import sys
print(sys.version)
%load_ext autoreload
%autoreload 2

3.9.5 (default, May 18 2021, 12:31:01) 
[Clang 10.0.0 ]


In [2]:
from datetime import date
import numpy as np

## Code questions

In [3]:
from utils import download_to_path

In [4]:
from utils import write_csv_row

In [6]:
import cv2

None

# Data mining

In [7]:
import database_classes
from naip_dbm import NAIPData
from numpy.random import uniform

In [8]:
from multidb_wrapper import GetMultiDBData

In [9]:
from shapely.geometry import Point

In [25]:
lat = 44.967243
lon = -103.771556

# make 10 random points within a given range
pont = Point(float(lon),float(lat))
points = [Point(float(lon+uniform(-.3,.3)),float(lat+uniform(-.3,.3))) for i in range(5)]

In [11]:
# naip = NAIPData(directory="banasna", features=["rgb", "ir"])

In [12]:
# naip.run(points)

In [13]:
# GetMultiDBData(NAIPData(features=["rgb", "ir"]))

In [19]:
gerri = GetMultiDBData(NAIPData(features=["rgb", "ir"]))

The default online source `{self.base_url}` will be used instead.


# Attention, currently, we are overwriting the files when we enter new locations!

In [26]:
gerri.run(points)

As no dates were specified the most recent data will be fetched.
logged in
v002/sd/2018/sd_060cm_2018/44103/m_4410303_nw_13_060_20180731_20181212.tif
https://naipblobs.blob.core.windows.net/naip
https://naipblobs.blob.core.windows.net/naip/v002/sd/2018/sd_060cm_2018/44103/m_4410303_nw_13_060_20180731_20181212.tif
/var/folders/h_/fp7vzm1s46x0fbjfhmjqmlsc0000gn/T/data_barrel/naip_western_europe_azure/source/v002/sd/2018/sd_060cm_2018/44103/m_4410303_nw_13_060_20180731_20181212.tif
Data from https://naipblobs.blob.core.windows.net/naip/v002/sd/2018/sd_060cm_2018/44103/m_4410303_nw_13_060_20180731_20181212.tif was retrieved to /var/folders/h_/fp7vzm1s46x0fbjfhmjqmlsc0000gn/T/data_barrel/naip_western_europe_azure/source/v002/sd/2018/sd_060cm_2018/44103/m_4410303_nw_13_060_20180731_20181212.tif.
RGB image for tile at location POINT (-103.7266113312103 44.98390909406123) was successfully downloaded to /var/folders/h_/fp7vzm1s46x0fbjfhmjqmlsc0000gn/T/data_barrel/naip_western_europe_azure/rgb/2

Perfect, there is one place with a large overlap, that could be used for trying the stitch protocoll:
45.07836209402709°N 104.0611188970986°W (`POINT (-104.0611188970986 45.07836209402709)`)
on google maps [here](https://www.google.at/maps/place/45%C2%B004'42.1%22N+104%C2%B003'40.0%22W/@45.0784439,-104.0619828,329m/data=!3m1!1e3!4m5!3m4!1s0x0:0x0!8m2!3d45.0783621!4d-104.0611189).
All the the files can be found in `/var/folders/h_/fp7vzm1s46x0fbjfhmjqmlsc0000gn/T/data_barrel/naip_western_europe_azure/source/v002/mt/2019/mt_60cm_2019/45104/` and are overlapping. we transfer them to the local dir.

In [10]:
target_point = Point(-104.0611188970986, 45.07836209402709)

In [63]:
class FileStitcher:
    '''
    Assembler of multiple files to one tile of given
    edgelength in pixel centered around a location.
    '''
    def __init__(self, location, tile_size_in_pixels,
                features):
        '''
        Construct with locations (shapely.geometry.Point) and 
        tile sizes given.
        '''
        self.location = location
        self.tile_size = tile_size_in_pixels
        self.features = features
        return
    
    
    def stitch_image(self, list_of_images: list):
        for image in list_of_images:
            make_temp_image(self, image_path)
        return
    
    
    def make_temp_image(self, image_path):
    '''
    Extract the focal tile (often partial) from a given image and store it.
    '''
    from fiona.transform import transform
    import numpy as np
    import rasterio

    IO_CRS = "epsg:4326"

    half_edge = int(self.tile_size/2)
    file_name = rawdata_path.replace(".tif", "rgb.tif")
    
    with rasterio.open(image_path) as image:
        # here coordinates need to be shifted from the input crs to
        # the tif image crs and later converted to pixel
        location_in_img_crs = [p[0] for p in transform(
            IO_CRS, image.crs.to_string(),
            [self.location.x], [self.location.y])]
        location_in_img_pix = [
            int(np.floor(p)) for p in
            ~image.transform * location_in_img_crs]

        # a window around the tile can be produced
        borders = [int(np.floor(p)) for p in
            ~image.transform * [image.bounds[2], image.bounds[1]]]
        
        # to avoid cutting a too large window, we define some edges
        # and lengths that help orienting
        wdw_left_edge = max([0, location_in_img_pix[0]-half_edge])
        wdw_top_edge = max([0, location_in_img_pix[1]-half_edge])
        wdw_right_edge = max([0, location_in_img_pix[0]+half_edge])
        wdw_bottom_edge = max([0, location_in_img_pix[1]+half_edge])

        wdw_height = min(
            [borders[1] - wdw_top_edge,  # dist btwn leftbrd and top wdw edge  
             self.tile_size,  # tile size
             wdw_bottom_edge]) # dist btwn rightbrd and bottom wdw edge  
        wdw_width = min(
            [borders[0] - wdw_left_edge,
             self.tile_size, 
             dw_right_edge])

        wdw = rasterio.windows.Window(
            wdw_left_edge,
            wdw_top_edge,
            wdw_width,
            wdw_height)
        image_tile = image.read(window=wdw)
        kwargs = image.meta.copy()
        kwargs.update({
            'height': wdw.height,
            'width': wdw.width,
            'transform': rasterio.windows.transform(wdw, image.transform)})
            # store the tiles in rgb or ir image, if requested
        images_produced = {}
    for profile, phot_prof in zip(["rgb", "ir"], ["RGB", "Grayscale"]):
        if True: # profile in feature.keys()
            try:
                with rasterio.open(
                        file_name.replace("rgb", profile), "w",
                        photometric=phot_prof, **kwargs) as file:
                    if profile == "rgb":
                        file.write(image_tile)
                    else:  # TODO: how to save as greyscale image?
                        file.write(image_tile[3] , indexes=1)  # we only store the last layer.
            except IOError as err:
                pass
            else:
                images_produced[profile] = file_name.replace("rgb", profile)
    return images_produced
        
        

IndentationError: expected an indented block (<ipython-input-63-677747fd4e6d>, line 18)

In [47]:
rawdata_path = "m_4510464_ne_13_060_20190913.tif"

file_name = rawdata_path.replace(".tif", "rgb.tif")
tile_size_pixels = 5000
location = target_point

In [54]:
a = cv2.imread("m_4510464_ne_13_060_20190913rgb.tif")
b = cv2.imread("m_4510464_nw_13_060_20190913rgb.tif")

In [ ]:
stitcher = cv2.Stitcher_create()

In [55]:
(status, stitched) = stitcher.stitch([a,b])

In [56]:
status

0

In [59]:
cv2.imwrite("stitched.tif", stitched)

True

ValueError: Source shape (4986, 4994, 3) is inconsistent with given indexes 4

## NAIP

<div class="alert alert-block alert-danger">
    <b>What exactly is this <a href="https://planetarycomputer.microsoft.com/dataset/naip#Blob-Storage-Notebook">Azure blob storage</a>?</b> <br> It seems, as if it does not need authentication, but we would, if we used <a href="https://planetarycomputer.microsoft.com/dataset/naip#Example-Notebook">Planetary Computer STAC API</a> by Microsoft or even Google's <a href="https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ#description">Earth Engine Data Catalog</a>. Where is the caveat? 
</div>

**Answer:** The explanation was too long to reconstruct for me. But the blob storage is really a path to circumvent the access abrrier in this case.

<div class="alert alert-block alert-danger">
    <b>Are the NAIP tiles actually overlapping?</b> <br> I am unsure, whether a location query could lead to multiple tiles within a given year.
</div>

In [6]:
multidb_wrapper.set_directory("aa")

'aa/data_barrel'

In [20]:
multidb_wrapper.set_locations([1,2,2,3], [2,3,4])

AssertionError: The lists for latitudes ({len(longitudes)}) and lon (4) have unequal length. Please adjust.

---

In [47]:


# code is again adjusted from ref01 - I dropped tom augspurger a mail
location = pont
tile_size_pixels = 500
query_url = "https://naipblobs.blob.core.windows.net/naip/v002/or/2016/or_100cm_2016/44118/m_4411830_se_11_1_20160721.tif"
file_name = "ahka_rgb.tif"
features = {"rgb": True, "ir": True}
def fetch_best_tile(location, tile_size_pixels, query_url,
                    file_name, features, silent=False):
    '''
    Download a tile of tile_size_pixels x tile_size_pixels of NAIP
    image centered around location, retrieved from query_url and
    to be stored as file_name.
    '''
    from fiona.transform import transform
    import numpy as np
    import rasterio

    IO_CRS = "epsg:4326"

    half_edge = int(tile_size_pixels/2)
    with rasterio.open(query_url) as image:
        # here coordinates need to be shifted from the input crs to
        # the tif image crs and later converted to pixel
        location_in_img_crs = [p[0] for p in transform(
            IO_CRS, image.crs.to_string(), [location.x], [location.y])]
        location_in_img_pix = [
            int(np.floor(p)) for p in
            ~image.transform * location_in_img_crs]

        # a window around the tile can be produced
        wdw = rasterio.windows.Window(
            location_in_img_pix[0]-half_edge, location_in_img_pix[1]-half_edge,
            tile_size_pixels, tile_size_pixels)
        image_tile = image.read(window=wdw)

        print(image_tile.shape)
        kwargs = image.meta.copy()
        kwargs.update({
            'height': wdw.height,
            'width': wdw.width,
            'transform': rasterio.windows.transform(wdw, image.transform)})
        
        # store the tiles in rgb or ir image, if requested
        for profile, phot_prof in zip(["rgb", "ir"], ["RGB", "Grayscale"]):
            if features[profile]:
                try:
                    with rasterio.open(file_name.replace("rgb", profile), "w",
                                       photometric=phot_prof, **kwargs) as file:
                        if profile == "rgb":
                            file.write(image_tile)
                        else:  # TODO: how to save as greyscale image?
                            file.write(image_tile[3], indexes=1)  # we only store the last layer.
                            # BUG - TODO: SOMEHOW THE IMAGE is only half but strechted
                except IOError:
                    print(f"It was not possible to store the {profile.upper()} image for tile at"
                          f" location {location}.\n" if not silent else "", end="")
                else:
                    print(f"{profile.upper()} image for tile at location {location} was "
                          f"successfully downloaded to {file_name}.\n" if not silent else "",
                         end="")
    return

<div class="alert alert-block alert-danger">
    <b>How can I see if my image was stored properly in 4 layers?</b> <br> When saving as tif and switching the axes, do I loose a layer?
</div>

In [48]:
fetch_best_tile(location, tile_size_pixels, query_url,
                    file_name, features)

[[ 52  57  60 ... 103  78  75]
 [ 41  44  45 ...  77  51  48]
 [ 41  40  34 ...  80  64  60]
 ...
 [ 48  58  79 ...  51  63  77]
 [ 59  59  66 ...  40  41  54]
 [ 66  68  59 ...  33  35  37]]
[[ 60  63  62 ... 103  83  73]
 [ 53  54  55 ...  88  61  57]
 [ 52  49  51 ...  80  68  65]
 ...
 [ 65  71  83 ...  76  75  78]
 [ 81  83  80 ...  63  58  64]
 [ 96  93  81 ...  54  52  49]]
[[68 69 71 ... 86 80 80]
 [65 65 69 ... 80 74 71]
 [67 65 66 ... 77 74 72]
 ...
 [67 71 73 ... 72 76 75]
 [74 72 72 ... 65 68 70]
 [75 73 70 ... 64 64 65]]
[[ 61  65  76 ... 156 112  96]
 [ 54  52  54 ... 123  64  61]
 [ 56  54  55 ... 101  69  67]
 ...
 [132 128 143 ... 162 132 106]
 [173 171 161 ... 134  96  67]
 [187 187 174 ...  91  79  53]]
(4, 500, 500)
RGB image for tile at location POINT (-118.25698 44.54595) was successfully downloaded to ahka_rgb.tif.
IR image for tile at location POINT (-118.25698 44.54595) was successfully downloaded to ahka_rgb.tif.
